In [1]:
import re

# short example for our precentage logic

exm_1_string = "The GC-content is 50%, and the pH is 7"
matches = re.findall(r"\%", exm_1_string)
if(len(matches) == 1):
    print("String contains exactly one %")


String contains exactly one %


In [19]:
from Bio import Entrez

Entrez.email = "bene6@hotmail.de"  # Always tell NCBI who you are

abstracts_with_one_perc = []

# Search PubMed for articles with "GC-content" in the title
for i in range(0, 400, 20):
    handle = Entrez.esearch(db="pubmed", term='("GC content of" OR "G + C content of") AND bacterial[Title/Abstract]', retstart=i, retmax=20)
    record = Entrez.read(handle)
    ids = record["IdList"]
    # Process the IDs

    # Get the list of article IDs
    article_ids = record["IdList"]

    # Fetch the abstracts for the articles
    handle = Entrez.efetch(db="pubmed", id=article_ids, rettype="abstract")
    records = Entrez.read(handle)

    # Print the abstracts
    noAbstract = 0
    for record in records["PubmedArticle"]:
        try:
            temp_record = record["MedlineCitation"]["Article"]
            matches = re.findall(r"\%", temp_record["Abstract"]["AbstractText"][0])
            if(len(matches) == 1):
                abstracts_with_one_perc.append(temp_record)
            pass
        except KeyError:
            print("No abstract available")
            noAbstract += 1
    print("No abstract available for " + str(noAbstract) + " articles "  + "by total of " + str(len(article_ids)) + " articles")


No abstract available
No abstract available for 1 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available
No abstract available for 1 articles by total of 20 articles
No abstract available for 0 articles by t

In [20]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]

'<i>Vibrio parahaemolyticus</i> is a major foodborne pathogen and the main cause of diarrheal diseases transmitted by seafood such as fish, shrimp, and shellfish. In the current study, a novel lytic phage infecting <i>V. parahaemolyticus</i>, vB_VpaP_GHSM17, was isolated from the sewage of a seafood market, Huangsha, Guangzhou, and its morphology, biochemistry, and taxonomy features were identified. Morphological observation revealed that GHSM17 had an icosahedral head with a short, non-contractile tail. The double-stranded DNA genome of GHSM17 consisted of 43,228 bp with a GC content of 49.42%. In total, 45 putative ORFs were identified in the GHSM17 genome. Taxonomic analysis indicated GHSM17 belonging to genus <i>Maculvirus</i>, family <i>Autographiviridae.</i> In addition, GHSM17 was stable over a wide range of temperatures (20-60 °C) and pH (5-11) and was completely inactivated after 70 min of ultraviolet irradiation. The bacterial inhibition assay revealed that GHSM17 could inhib

In [21]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"].keys()
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["PublicationTypeList"]
# records["PubmedArticle"][0]["MedlineCitation"].keys()
# records["PubmedArticle"][0]["MedlineCitation"].keys()
# records["PubmedArticle"][0].keys()


# records["PubmedArticle"][0]["PubmedData"].keys()

[StringElement('Journal Article', attributes={'UI': 'D016428'}), StringElement("Research Support, Non-U.S. Gov't", attributes={'UI': 'D013485'})]

In [22]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"]

[StringElement('1601', attributes={'EIdType': 'pii', 'ValidYN': 'Y'}),
 StringElement('10.3390/v14081601', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})]

In [23]:
# abstracts_with_one_perc[0]
matches = re.findall(r"\%", abstracts_with_one_perc[0]["Abstract"]["AbstractText"][0])
if(len(matches) > 1):
    print("String contains more than one percent symbol")

In [24]:
len(abstracts_with_one_perc)

73

In [25]:
import pandas as pd
abstracts_with_one_perc[0]

pd.DataFrame([x["Abstract"]["AbstractText"] for x in abstracts_with_one_perc]).to_csv("test.csv", index=None)

In [26]:
hopefull_dicts = []

# give the one <i> tag which is the nearest to the % sign

# Add DOI to the dict
# Add Abstract to the dict
# ETC


for pre_x in abstracts_with_one_perc:
    x = pre_x["Abstract"]["AbstractText"][0]
    
    # Extract the content between the tags
    result = re.search(r"<i>(.*?)</i>", x)
    if result:
        hopefully_bacterial = result.group(1)
    else:
        hopefully_bacterial = None


    # Extract the precentage value
    
    # give me everything which is close to the % sign
    result = re.search(r"(.{3}\%.{3})", x)
    if result:
        hopefully_percentage_for_bacterial =str('" ' + result.group(1) + ' "')
    else:
        hopefully_percentage_for_bacterial = None

    is_valuable_data = ""
    print("Is the data for " + str(hopefully_bacterial) + " with " + str(hopefully_percentage_for_bacterial) + " correct? (y/n/m)")
    
    while(is_valuable_data != "y" and is_valuable_data != "n" and is_valuable_data != "m"):
        is_valuable_data = input("Is the data for " + str(hopefully_bacterial) + " with " + str(hopefully_percentage_for_bacterial) + " correct? (y/n/m)")

    hopefull_dict = {hopefully_bacterial: hopefully_percentage_for_bacterial, "is_valuable_data": is_valuable_data}

    hopefull_dicts.append(hopefull_dict)

# show only dicts with a value and a key
# hopefull_dicts = [x for x in hopefull_dicts if list(x.keys())[0] is not None and list(x.values())[0] is not None]
hopefull_dicts
# how do we verify?





Is the data for Variovorax with " 70)%, r " correct? (y/n/m)
Is the data for Psychrobacter with " 4.1% G+ " correct? (y/n/m)
Is the data for None with None correct? (y/n/m)
Is the data for Roseiflexus with " 0.5% re " correct? (y/n/m)
Is the data for None with None correct? (y/n/m)
Is the data for Myxococcus with " 7.5% an " correct? (y/n/m)
Is the data for Sphingomonas with " .1 %, r " correct? (y/n/m)


KeyboardInterrupt: Interrupted by user

In [19]:
# only show the ones which are valuable
[x for x in hopefull_dicts if x["is_valuable_data"] == "m"]

[{'amoA': '2%', 'is_valuable_data': 'm'},
 {'Caragana microphylla, Caragana liouana': '1%', 'is_valuable_data': 'm'},
 {'E. coli': '7.4%', 'is_valuable_data': 'm'},
 {'Haliclona (Rhizoniera)': '80%', 'is_valuable_data': 'm'},
 {'Oceanobacillus jordanicus': '39.09%', 'is_valuable_data': 'm'},
 {'Xanthomonas oryzae': '63.7%', 'is_valuable_data': 'm'},
 {'Bacillus': '41.41%', 'is_valuable_data': 'm'},
 {'Streptomyces': '71.80%', 'is_valuable_data': 'm'},
 {'Paenibacillus': '46%', 'is_valuable_data': 'm'},
 {'Enterobacter roggenkampii': '56.16%', 'is_valuable_data': 'm'},
 {'Pseudarthrobacter phenanthrenivorans': '65.30%', 'is_valuable_data': 'm'}]

In [14]:
import pandas as pd
# read csv Simulation.csv 


random_df = pd.read_csv("Simulation.csv", sep=";")

In [16]:
random_df.to_csv("Simulation_good_csv.csv", sep=",", index=False, quoting=1)

In [27]:
# start python script in git bash python "C:\OneDrive\Technische Hochschule\DIS18a-b\repo\DIS18a-b-Group-4\src\script/test_python.py"
